In [4]:
import numpy as np
import matplotlib.pyplot as plt
from xxhash import xxh32
import B2D
from skimage import io

### APPM_RT(B)
    Input:  B 進制
    Return: APPM_reference_table in base B

In [7]:
def APPM_RT(B):
    x = int(np.sqrt(B))
    small_RT = np.array(np.random.choice(range(B), B, replace=False)).reshape(x, x)
    RT = np.zeros((B, B), dtype=int)  
    for i in range(0, B, x):
        for j in range(0, B, x):
            RT[i:i+x, j:j+x] = small_RT
    return RT

In [9]:
table = APPM_RT(256)

### Queue()
    Input:  Null
    Return: empty queue

In [29]:
class Queue():
    def __init__(self):
        self.q = []
        self.r = -1
    def reset(self):
        self.q = []
        self.r = -1    
    def enqueue(self,r,c):
        self.q.append((r,c))
    def dequeue(self):
        self.r += 1
        if self.r >= len(self.q):
            return False
        return self.q[self.r]

### BFS(RT,r,c,g)
    Input:  reference table ,row ,column ,goal
    Return: the nearest goal from the starting position

In [30]:
def BFS(RT,r,c,g):
    Q = Queue()
    Q.enqueue(r,c)
    F = np.full((RT.shape[0],RT.shape[1]),False,dtype=bool)
    F[r,c] = True
    while True:
        k = Q.dequeue()
        if k == False:
            break
        if RT[k[0],k[1]] == g:
                return k
        if k[0] != 0 and not F[k[0]-1,k[1]]:
            Q.enqueue(k[0]-1,k[1])
            F[k[0]-1,k[1]] = True
        if k[0] != RT.shape[0]-1 and not F[k[0]+1,k[1]]:
            Q.enqueue(k[0]+1,k[1]) 
            F[k[0]+1,k[1]] = True
        if k[1] != 0 and not F[k[0],k[1]-1]:
            Q.enqueue(k[0],k[1]-1)
            F[k[0],k[1]-1] = True
        if k[1] != RT.shape[1]-1 and not F[k[0],k[1]+1]:
            Q.enqueue(k[0],k[1]+1)  
            F[k[0],k[1]+1] = True  

### hashB(npArray,bits):
    Input:  npArray(key), how many bits will return
    Return: some bits

In [31]:
def hashB(npArray,bits):
    return np.mod(xxh32(npArray).intdigest(), 2**bits)

### def fold(Dec_num,bit):
    Input:  Dec_num (decimal number), bit (number of bits to return)
    Return: folded binary representation with the specified number of bits
    Info:   convert the decimal number to binary, then fold the binary array into the specified number of  bits

In [32]:
def fold(Dec_num,bit):
    k = B2D.Dec2Bin(Dec_num)
    l = len(k)
    while(l > bit):
        l >>= 1
        a = k[:l]
        b = k[l:]
        k = []
        for i in range(l):
            k.append(a[i]^b[i])
    return B2D.Bin2Dec(k)

### def perturbed(r, g, a):
    Input:  r (red value), g (green value), a (amplitude of perturbation)
    Return: array of possible perturbed values for (r, g)
    Info:   generates all possible (r, g) pairs by perturbing the original values within the range [-a, a) while keeping the values within valid RGB bounds (0 to 255)



In [33]:
def perturbed(r,g,a):
    possible = []
    for i in range(-1*a,a):
        for j in range(-1*a,a):
            if(not (r+i < 0 or r+i > 255 or g+j < 0 or g+j > 255)):
                possible.append([r+i,g+j])
    return np.array(possible)

### def remedy(r, g, b, ac):
    Input:  r (red value), g (green value), b (blue value), ac (verification code)
    Return: modified RGB values [r', g', b'] that match certain grayscale constraints
    Info:   the function first computes the grayscale value based on the input RGB values. Then, it perturbs the red and green values within a small range using the `perturbed()` function, and adjusts the blue value by rounding it to a multiple of 4. The function then checks for the combination of red, green, and blue values that maintains the same grayscale value for two specific calculations (with and without adding 3 to the blue component). Once a valid combination is found, it adjusts the blue value based on `ac` (verification code) and returns the RGB combination that minimizes the distance from the original input.


In [34]:
def remedy(r,g,b,ac):
    Gray = round(r*0.299+g*0.587+b*0.114)
    rg_poss = perturbed(r,g,2)
    b_poss = []
    origin_b = b
    b >>= 2
    b <<= 2
    for i in range(-1,2):
        if(b + i*4 >= 0 and b + i*4 <= 255):
            b_poss.append(b + i*4)

    b_poss = np.array(b_poss)

    tar = None
    distance = np.inf
    for i in range(len(rg_poss)):
        for j in range(len(b_poss)):
            if(Gray == round(rg_poss[i,0]*0.299+rg_poss[i,1]*0.587+b_poss[j]*0.114) and
               Gray == round(rg_poss[i,0]*0.299+rg_poss[i,1]*0.587+(b_poss[j]+3)*0.114)):
                b_poss[j] >>= 2
                b_poss[j] <<= 2
                b_poss[j] += ac
                
                new_d = ((rg_poss[i,0]-r)**2 + (rg_poss[i,1]-g)**2 + (b_poss[j]-origin_b)**2)**0.5

                if(new_d < distance):
                    distance = new_d
                    tar = [rg_poss[i,0],rg_poss[i,1],b_poss[j]]

    return tar

### def AVGI(Graph):
    Input:  Graph (image filename)
    Return: PSNR value, count of outliers, and saves the modified image
    Info:   The function embeds a verification code (grayscale value) into the red (R) and blue (B) channels of an image using the APPM (Adaptive Pixel Pair Matching) method. Instead of directly embedding the code into the least significant bits (LSB), it adjusts the R and B values by modifying their positions in the APPM reference table to embed the grayscale value as a verification code. The green channel (G) is adjusted to maintain the grayscale consistency, ensuring that the pixel remains as close as possible to the original grayscale value. The function calculates the PSNR (Peak Signal-to-Noise Ratio) to assess the image quality and saves the modified image.

    Detailed Steps:
    1. Load the image from the path `'image/'+Graph+'.tiff'`.
    2. Copy the image to `Stego` to create a modified version for embedding the code.
    3. Initialize the APPM reference table using `APPM_RT(256)`.
    4. Iterate through each pixel of the image:
       - Compute the grayscale value using the formula `Gray = 0.299 * R + 0.587 * G + 0.114 * B`.
       - Round the grayscale value to the nearest integer and use this as the verification code (`ac`).
       - Use the `BFS()` function to find the nearest R and B values in the APPM reference table for embedding the verification code.
       - Adjust the green channel (`g_bar`) to maintain the grayscale consistency based on the modified R and B values.
       - Handle edge cases where `g_bar` exceeds the valid RGB range (0-255) by adjusting `g_bar` and tracking the number of outliers (`p`).
    5. Calculate the mean squared error (MSE) between the original and modified pixel values.
    6. Compute the PSNR to evaluate the quality of the stego image.
    7. Save the modified image and write the PSNR and outlier count to a text file.
    8. Display the modified image using `io.imshow()` and save it as a PNG.


In [35]:
def AVGI(Graph):
    path = 'image/'+Graph
    I=io.imread(path +r'.tiff')
    Stego = I.copy()
    RT = APPM_RT(256)

    p = 0
    MSE = 0

    for i in range(Stego.shape[0]):
        for j in range(Stego.shape[1]):
            Gray = I[i,j,0]*0.299+I[i,j,1]*0.587+I[i,j,2]*0.114
            G_round = round(Gray)
            ac = G_round    
            
            k = BFS(RT,Stego[i,j,0],Stego[i,j,2],ac)          

            Stego[i,j,0] = k[0]
            Stego[i,j,2] = k[1]  
            g_bar = int((Gray - 0.299*k[0] - 0.114*k[1])/0.587)
            if(round(0.299*k[0]+0.587*g_bar+0.114*k[1]) < round(Gray)):
                g_bar += 1
            elif(round(0.299*k[0]+0.587*g_bar+0.114*k[1]) > round(Gray)):
                g_bar -= 1

            if(g_bar > 255):
                p += 1
                g_bar = 510 - abs(int((G_round - 0.299*k[0] - 0.114*k[1])/0.587))
            elif(g_bar < 0):
                p += 1
                g_bar = abs(int((G_round - 0.299*k[0] - 0.114*k[1])/0.587))
                
            Stego[i,j,1] = g_bar

            delta = int(Stego[i,j,2]) - int(I[i,j,2])
            MSE += delta ** 2
            delta = int(Stego[i,j,1]) - int(I[i,j,1])
            MSE += delta ** 2 
            delta = int(Stego[i,j,0]) - int(I[i,j,0])           
            MSE += delta ** 2                                        

    MSE /= (Stego.shape[0]*Stego.shape[1]*3)
    PSNR = 10 * np.log10(65025/MSE)
    print(f"PSNR:{PSNR} , F:{p}")

    with open("processing_data/"+Graph+".txt","w") as file:
        file.write(f"PSNR: {PSNR}\n")
        file.write(f"outliers: {p}")


    io.imshow(Stego)
    io.show()
    io.imsave('processing_image/'+Graph+'.png',Stego)

### def Authorize(Graph):
    Input:  Graph (image filename)
    Return: None (displays the verification result and modified image)
    Info:   The function checks if the image has been tampered with by verifying the embedded grayscale value (used as a verification code) in the red (R) and blue (B) channels. The grayscale value is derived from the APPM (Adaptive Pixel Pair Matching) reference table. The function compares the stored grayscale value with the one reconstructed from the R and B values. If there is a mismatch, the function first checks whether the green (G) value adheres to the foldback rule. For example, if the required green value (G) to balance the red (R) and blue (B) values is negative (e.g., G = -3), the green value is folded back to its positive equivalent (G = 3) to maintain a valid RGB range. Only if the green value does not satisfy the foldback rule, the pixel is marked as tampered by setting its RGB values to white (255, 255, 255). Otherwise, if the green value after foldback still maintains the intended grayscale balance, the pixel is considered untampered and marked as black (0, 0, 0). This ensures that some natural adjustments (like green foldback) are allowed without flagging the pixel as tampered, while actual mismatches that cannot be corrected are flagged as tampered.



    Detailed Steps:
    1. Load the image from the path `'embeding_noise/'+Graph+'.png'`.
    2. Copy the image to `Stego` to check the integrity of the embedded grayscale value.
    3. Initialize the APPM reference table using `APPM_RT(256)`.
    4. Iterate through each pixel of the image:
       - Compute the grayscale value using `Gray = 0.299 * R + 0.587 * G + 0.114 * B`.
       - Round the grayscale value to the nearest integer and use it as the verification code (`ac`).
       - Compare the stored verification code with the one derived from the APPM reference table.
       - If a mismatch is found, mark the pixel as tampered by setting its RGB values to (255, 255, 255).
       - If no mismatch is found, mark the pixel as untampered by setting its RGB values to (0, 0, 0).
       - Print the details of tampered pixels including their coordinates and values.
    5. Display the modified image using `io.imshow()`.
    6. Print a message indicating whether the image has been tampered with.


In [36]:
def Authorize(Graph):
    path = "embeding_noise/"+Graph+".png"

    I=io.imread(path)
    Stego = I.copy()
    RT = APPM_RT(256)
    print(Stego.shape[2])
    Flag = False

    for i in range(Stego.shape[0]):
        for j in range(Stego.shape[1]):
            Gray = I[i,j,0]*0.299+I[i,j,1]*0.587+I[i,j,2]*0.114
            G_round = round(Gray)
            ac = G_round
            flag = False
            if(ac > RT[Stego[i,j,0],Stego[i,j,2]]):
                ac = abs(int((RT[Stego[i,j,0],Stego[i,j,2]] - 0.299*Stego[i,j,0] - 0.114*Stego[i,j,2])/0.587))
                if(Stego[i,j,1] != ac):
                    Stego[i,j,0] = 255
                    Stego[i,j,1] = 255
                    Stego[i,j,2] = 255
                    flag = True
                    print(f"This picture is tampered. i: {i} ,j: {j} ,Stego:{Stego[i,j]} ,Gray:{Gray}, RT:{RT[Stego[i,j,0],Stego[i,j,2]]}")
                    Flag = True
            elif(ac < RT[Stego[i,j,0],Stego[i,j,2]]):
                ac = 510 - abs(int((RT[Stego[i,j,0],Stego[i,j,2]] - 0.299*Stego[i,j,0] - 0.114*Stego[i,j,2])/0.587))
                if(Stego[i,j,1] != ac):                
                    Stego[i,j,0] = 255
                    Stego[i,j,1] = 255
                    Stego[i,j,2] = 255
                    flag = True
                    print(f"This picture is tampered. i: {i} ,j: {j} ,Stego:{Stego[i,j]} ,Gray:{Gray}, RT:{RT[Stego[i,j,0],Stego[i,j,2]]}")
                    Flag = True
            if(not flag):
                Stego[i,j,0] = 0
                Stego[i,j,1] = 0
                Stego[i,j,2] = 0
    io.imshow(Stego, vmin=0, vmax=255)
     
    plt.show()  # 不调用 tight_layout
    if(not Flag):
        print("This picture is not tampered.")